# ACC with stop&go

In [1]:
import carla, time, pygame, math, random, cv2
import open3d as o3d
from matplotlib import cm
import numpy as np

pygame 2.6.1 (SDL 2.28.4, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
spectator = world.get_spectator()

## Some helpful functions used in this notebook

In [3]:
def move_spectator_to(transform, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    
    spectator_transform = carla.Transform(back_location, transform.rotation)
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*', transform = carla.Transform()):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    spawn_point.location.x += transform.location.x
    spawn_point.location.y += transform.location.y
    spawn_point.location.z += transform.location.z
    spawn_point.rotation.pitch += transform.rotation.pitch
    spawn_point.rotation.roll += transform.rotation.roll
    spawn_point.rotation.yaw += transform.rotation.yaw
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def draw_on_screen(world, transform, content='O', color=carla.Color(0, 255, 0), life_time=20):
    world.debug.draw_string(transform.location, content, color=color, life_time=life_time)

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), width=800, height=600):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    #camera_bp.set_attribute('fov', '120')
    #camera_bp.set_attribute('sensor_tick', '0')
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

def spawn_radar(attach, transform=carla.Transform(carla.Location(x=1.2, z=1.2)), range=100, points_per_second=10):
    radar_bp = world.get_blueprint_library().find('sensor.other.radar')
    radar_bp.set_attribute('horizontal_fov', '30')
    radar_bp.set_attribute('vertical_fov', '30')
    #radar_bp.set_attribute('range', str(100.0))
    print("sdhgakjfhjd")
    print(type(radar_bp.get_attribute('range')))
    print(radar_bp.get_attribute('range'))
    radar_bp.set_attribute('range', str(range))
    print(type(radar_bp.get_attribute('range')))
    print(radar_bp.get_attribute('range'))
    #radar_bp.set_attribute('points_per_second', str(points_per_second))
    radar = world.spawn_actor(radar_bp, transform, attach_to=attach)
    return radar

#TTC = (distance / relative_velocity) if relative_velocity != 0 else 9999
def type_detection(data) -> carla.RadarDetection:
    return data
    

# Simple callback function to print the number of detections
def handle_measurement(data: carla.RadarMeasurement, vehicle: carla.Vehicle):
    print(len(data))
    print(data.get_detection_count())
    for m in data:
        newM = type_detection(m)
        print("data: ",newM)
        print("DIO LAIDO ", newM.depth)
    

## Sensors

Sensors are another type of actors, usually spawned attached to a vehicle, designed to retrieve data from the world. The type of data depends on the sensor, but it can range from RGB images, LiDAR scans or even collision information.

Sensors are divided into two main categories:

- **Regular sensors**: these sensors retrieve data at a fixed rate, usually every tick (e.g RGB cameras).
- **Trigger sensors**: these sensors only retrieve data when a certain condition is met (e.g collision sensors).

Sensors details can be found in the [CARLA documentation](https://carla.readthedocs.io/en/latest/ref_sensors/).

![Sensors](img/sensors.jpg)

### Live camera feed

We can also visualize the camera feed in real-time. This is useful for debugging and understanding what the camera sees.

We can use the `opencv` library to create a window and display the camera feed.

In [5]:
VIRIDIS = np.array(cm.get_cmap('plasma').colors)
VID_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])

COOL_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])
COOL = np.array(cm.get_cmap('winter')(COOL_RANGE))
COOL = COOL[:,:3]

def add_open3d_axis(vis):
    axis = o3d.geometry.LineSet()
    axis.points = o3d.utility.Vector3dVector(np.array([
        [0.0, 0.0, 0.0],
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]]))
    axis.lines = o3d.utility.Vector2iVector(np.array([
        [0, 1],
        [0, 2],
        [0,3]]))
    axis.colors = o3d.utility.Vector3dVector(np.array([
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]]))
    vis.add_geometry(axis)

In [6]:
def radar_callback(data, point_list):
    radar_data = np.zeros((len(data), 4))

    for i, detection in enumerate(data):
        x = detection.depth * math.cos(detection.altitude) * math.cos(detection.azimuth)
        y = detection.depth * math.cos(detection.altitude) * math.sin(detection.azimuth)
        z = detection.depth * math.sin(detection.altitude)

        radar_data[i, :] = [x, y, z, detection.velocity]

    intensity = np.abs(radar_data[:, -1])
    intensity_col = 1.0 -np.log(intensity)/np.log(np.exp(-0.004 * 100))
    int_color = np.c_[
        np.interp(intensity_col, COOL_RANGE, COOL[:, 0]),
        np.interp(intensity_col, COOL_RANGE, COOL[:, 1]),
        np.interp(intensity_col, COOL_RANGE, COOL[:, 2])]  
    points = radar_data[:, :-1]
    points[:,:1] = -points[:,:1]
    point_list.points = o3d.utility.Vector3dVector(points)
    point_list.colors = o3d.utility.Vector3dVector(int_color)

def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

In [1]:

pygame.init()
# focus on the pygame window to get the key pressed
pygame.display.set_mode((400, 300))

for v in world.get_actors().filter('vehicle.*'):
    v.destroy()

ego_vehicle = spawn_vehicle()
radar = spawn_radar(attach=ego_vehicle)
#radar.listen(lambda data: handle_measurement(data, ego_vehicle))
vehicle2 = spawn_vehicle(transform=carla.Transform(carla.Location(x=50)))
camera = spawn_camera(attach_to=ego_vehicle, transform=carla.Transform(carla.Location(x=-6, z=5), carla.Rotation(pitch=-30)))

#point_list = o3d.geometry.PointCloud()
radar_list = o3d.geometry.PointCloud()

camera_data = {'image': np.zeros((800,600,4))}

#video_output = np.zeros((600, 800, 4), dtype=np.uint8)
#def camera_callback(image):
    #global video_output
    #video_output = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

#camera.listen(lambda image: camera_callback(image))
camera.listen(lambda image: camera_callback(image, camera_data))
radar.listen(lambda data: radar_callback(data, radar_list))
#vehicle.set_autopilot(True)
#control = carla.VehicleControl(throttle=1000)
#vehicle.apply_control(control)

cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])
cv2.waitKey(1)

vis = o3d.visualization.Visualizer()
vis.create_window(
    window_name ='Carla',
    width=960,
    height=540,
    left=480,
    top=270)

vis.get_render_option().background_color = [0.05, 0.05, 0.05]
vis.get_render_option().point_size = 1
vis.get_render_option().show_coordinate_frame = True
add_open3d_axis(vis)

frame = 0

running = True
moving_forward = False
move_spectator_to(ego_vehicle.get_transform())

try:
    while running:
        current_location = ego_vehicle.get_location()
        if frame == 2:
            #vis.add_geometry(point_list)
            vis.add_geometry(radar_list)
        #vis.update_geometry(point_list)
        vis.update_geometry(radar_list)
        try:
            vis.pool_events()
            vis.update_renderer()
        except:
            continue    
        time.sleep(0.005)
        frame+=1

        cv2.imshow('RGB Camera', camera_data['image'])    

        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                key = event.key
                if key == pygame.QUIT or key == pygame.K_ESCAPE:
                    running = False
                elif key == pygame.K_w:
                    current_location.x += 2
                    #world.tick()
                    moving_forward = True
                    ego_vehicle.apply_control(carla.VehicleControl(throttle=0.5))
                elif key == pygame.K_s:
                    current_location.x -= 2
                    ego_vehicle.apply_control(carla.VehicleControl(throttle=0.5, reverse=True))
                    #world.tick()
                    moving_forward = False

        
        
        #world.tick()
        
        pygame.display.flip()


        #print("Actor transform: ", ego_vehicle.get_transform())
        #print("Actor forward vector (direction):", ego_vehicle.get_transform().get_forward_vector())
        #print("Actor velocity: ", ego_vehicle.get_velocity(),", ",ego_vehicle.get_velocity().length(),"m/s, ",ego_vehicle.get_velocity().length()*3.6,"km/h")
        #cv2.imshow('RGB Camera', video_output)
        #move_spectator_to(ego_vehicle.get_transform())
except KeyboardInterrupt:
    pass 
finally:
    pygame.quit()
    cv2.destroyAllWindows()
    radar.destroy()
    camera.stop()
    camera.destroy()
    ego_vehicle.destroy()
    vehicle2.destroy()
    vis.destroy_window()

NameError: name 'pygame' is not defined

## SOME SHIT

In [ ]:
client = carla.Client('localhost', 2000)
print(client.get_available_maps())
client.load_world("Town05_Opt")
client.set_timeout(10.0)
time.sleep()
world = client.get_world()
spectator = world.get_spectator()

['/Game/Carla/Maps/Town01', '/Game/Carla/Maps/Town01_Opt', '/Game/Carla/Maps/Town02', '/Game/Carla/Maps/Town02_Opt', '/Game/Carla/Maps/Town03', '/Game/Carla/Maps/Town03_Opt', '/Game/Carla/Maps/Town04', '/Game/Carla/Maps/Town04_Opt', '/Game/Carla/Maps/Town05', '/Game/Carla/Maps/Town05_Opt', '/Game/Carla/Maps/Town10HD', '/Game/Carla/Maps/Town10HD_Opt']


In [ ]:
camera_transforms = [
    (carla.Transform(carla.Location(x=1.5, z=2.4)), (600, 300)),  # Front camera
    (carla.Transform(carla.Location(x=-0.5, y=-0.9, z=2.4), carla.Rotation(yaw=-135)), (200, 400)),  # Left side camera
    (carla.Transform(carla.Location(x=-0.5, y=0.9, z=2.4), carla.Rotation(yaw=135)), (200, 400)),  # Right side camera
    (carla.Transform(carla.Location(x=-1.5, z=2.4), carla.Rotation(yaw=180)), (600, 300))  # Rear camera
]

ego_vehicle = spawn_vehicle()
#control = carla.VehicleControl(throttle=0.5)
#vehicle.apply_control(control)
#world.tick()
ego_vehicle.set_autopilot(True)


# Spawn cameras and attach to vehicle
cameras = []
video_outputs = [np.zeros((600, 800, 4), dtype=np.uint8) for _ in range(4)]

def create_camera_callback(index):
    def camera_callback(image):
        global video_outputs
        video_outputs[index] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    return camera_callback

for i, transform in enumerate(camera_transforms):
    camera = spawn_camera(attach_to=ego_vehicle, transform=transform[0], width=transform[1][0], height=transform[1][1])
    camera.listen(create_camera_callback(i))
    cameras.append(camera)

cv2.namedWindow('Front Camera', cv2.WINDOW_AUTOSIZE)
cv2.namedWindow('Left Side Camera', cv2.WINDOW_AUTOSIZE)
cv2.namedWindow('Right Side Camera', cv2.WINDOW_AUTOSIZE)
cv2.namedWindow('Rear Camera', cv2.WINDOW_AUTOSIZE)

running = True

try:
    while running:
        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        cv2.imshow('Front Camera', video_outputs[0])
        cv2.imshow('Left Side Camera', video_outputs[1])
        cv2.imshow('Right Side Camera', video_outputs[2])
        cv2.imshow('Rear Camera', video_outputs[3])
finally:
    cv2.destroyAllWindows()
    for camera in cameras:
        camera.destroy()
    ego_vehicle.destroy()

In [ ]:
ego_vehicle = spawn_vehicle()

gps_blueprint = world.get_blueprint_library().find('sensor.other.gnss')
gps_sensor = world.spawn_actor(gps_blueprint, carla.Transform(), attach_to=ego_vehicle)
print(carla.Transform())

ROAD_X_MIN = -110
LONGITUDE_MAX = 0.001

latest_gps = {"latitude": 0.0, "longitude": 0.0}

def gps_callback(gps_data):
    global latest_gps
    latest_gps["latitude"] = gps_data.latitude
    latest_gps["longitude"] = gps_data.longitude

gps_sensor.listen(gps_callback)

try:
    while True:
        current_x = latest_gps["longitude"]
        print(f"Longitude: {current_x}", end='\r')

        if current_x > LONGITUDE_MAX:
            new_location = carla.Location(x=ROAD_X_MIN, y=ego_vehicle.get_transform().location.y)
            ego_vehicle.set_transform(carla.Transform(new_location, ego_vehicle.get_transform().rotation))

        control = carla.VehicleControl(throttle=0.5)
        ego_vehicle.apply_control(control)

        world.tick()
except KeyboardInterrupt:
    pass
finally:
    gps_sensor.destroy()
    ego_vehicle.destroy()

## WTF?!

In [8]:
#if is sync mode you have to tick for every action, in async mode tick is use to add some flow control aka request the sim to immediately execute command???    

world = client.get_world()
settings = world.get_settings()
print(settings)
#settings.synchronous_mode = True #to set syncronous mode
world.apply_settings(settings)
#client.reload_world()


for v in world.get_actors().filter('vehicle.*'):
    v.destroy()

ego_vehicle = spawn_vehicle()
world.tick()
radar = spawn_radar(attach=ego_vehicle, range=1)
radar.listen(lambda data: handle_measurement(data, ego_vehicle))
#ego_vehicle.set_autopilot(True) #using this instead of the apply_control() method requires world.tick()
ego_vehicle.apply_control(carla.VehicleControl(throttle=0.5))
running = True

move_spectator_to(ego_vehicle.get_transform())
world.tick()

try:
    while running:
        move_spectator_to(ego_vehicle.get_transform())
        #world.tick() #without this the spectator movement is very slow
except KeyboardInterrupt:
    pass        
finally:
    ego_vehicle.destroy() 


2
2
data:  RadarDetection(velocity=0.000000, azimuth=-0.010484, altitude=-0.245889, depth=4.925483)
DIO LAIDO  4.925482749938965
data:  RadarDetection(velocity=0.000000, azimuth=-0.039372, altitude=-0.251014, depth=4.826505)
DIO LAIDO  4.826505184173584
sdhgakjfhjd
<class 'carla.libcarla.ActorAttribute'>
ActorAttribute(id=range,type=float,value=100)
<class 'carla.libcarla.ActorAttribute'>
ActorAttribute(id=range,type=float,value=5)
0
0


In [ ]:
vehicle = spawn_vehicle(world)
time.sleep(1)
move_spectator_to(vehicle.get_transform(), spectator)

def random_control(vehicle):
    control = carla.VehicleControl()
    control.throttle = random.uniform(0.5, 1.0)
    control.steer = random.uniform(-1.0, 1.0)
    vehicle.apply_control(control)

try:
    while True:
        random_control(vehicle)
        world.tick() #why is needed??????????????? ensures that random-control is applied syncronously? 
        time.sleep(0.1)
except KeyboardInterrupt:
    pass
finally:
    vehicle.destroy()

In [ ]:
radar_bp.set_attribute('range', '20')           # Maximum range -> ??? il default è 100..